In [29]:
%qtconsole
import numpy as np
import pandas as pd
from service import files_service
from model.models import TradeModel
from sklearn.preprocessing import normalize

from keras.models import Sequential
from keras.layers.core import Dense, Activation,Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Set folder for data

In [2]:
PATH = "data/test_all_periods/"

In [3]:
files = files_service.get_files(PATH,extension='.csv')
files = [files[2]]

In [4]:
files

['EURUSD_UTC_4H_Bid_2010.01.01_2017.02.18.csv']

In [5]:
df = pd.read_csv("".join([PATH,files[0]]))

In [6]:
df.head()

,Time (UTC),Open,High,Low,Close,Volume
0,01.01.2010 08:00:00,1.43297,1.43346,1.43218,1.43306,18182.8
1,01.01.2010 12:00:00,1.43321,1.43354,1.43207,1.43337,15405.0
2,01.01.2010 16:00:00,1.43337,1.43356,1.43225,1.43342,13920.0
3,01.01.2010 20:00:00,1.43331,1.43356,1.43315,1.43335,6956.1
4,03.01.2010 20:00:00,1.43024,1.43359,1.42951,1.43141,3001.6


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11536 entries, 0 to 11535
Data columns (total 6 columns):
Time (UTC)    11536 non-null object
Open          11536 non-null float64
High          11536 non-null float64
Low           11536 non-null float64
Close         11536 non-null float64
Volume        11536 non-null float64
dtypes: float64(5), object(1)
memory usage: 540.8+ KB


In [8]:
TradeModel_dict={}
for f in files:
    name = f.split('.csv')[0]
    frequency =  '\"' + f.split('_')[2] + '\"'
    print name, frequency
    path = PATH+f
    tm = TradeModel(path,name,frequency=frequency)
    TradeModel_dict[name]= tm

EURUSD_UTC_4H_Bid_2010.01.01_2017.02.18 "4H"


In [9]:
tm =TradeModel_dict['EURUSD_UTC_4H_Bid_2010.01.01_2017.02.18']

In [10]:
tm.df.to_csv('tradeModel_data.csv')

In [11]:
df = tm.df

In [12]:
df.head()

,Open,High,Low,Close,Volume,KC_M_1,KC_U_1,KC_D_1,KC_M_2,KC_U_2,...,DOWN_next_Low_under_Close,DOWN_next_High_over_Close,UP_next_High_over_Close,UP_next_Low_under_Close,DOWN_next_Low_and_High,UP_next_Low_and_High,hour,day,month,year
date_time,,,,,,,,,,,,,,,,,,,,,
2010-01-01 08:00:00,1.43297,1.43346,1.43218,1.43306,18182.8,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,8,4,1,2010
2010-01-01 12:00:00,1.43321,1.43354,1.43207,1.43337,15405.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,12,4,1,2010
2010-01-01 16:00:00,1.43337,1.43356,1.43225,1.43342,13920.0,1.432990,1.434343,1.431637,1.432990,1.434343,...,0,0,0,0,0,0,16,4,1,2010
2010-01-01 20:00:00,1.43331,1.43356,1.43315,1.43335,6956.1,1.433141,1.434204,1.432078,1.433141,1.434204,...,1,0,0,1,0,0,20,4,1,2010
2010-01-03 20:00:00,1.43024,1.43359,1.42951,1.43141,3001.6,1.432644,1.434578,1.430711,1.432644,1.434578,...,1,0,0,1,0,0,20,6,1,2010


In [13]:
def train_test_split(tm, date, target):
    
    
              
    predictors =tm.get_predictors_columns()
    #predictors =['Open','High','Low','High','KC_M_3','KC_U_3','ATR','EMA12','EMA30']
    print tm.df.shape
    tm.df = tm.df.dropna()
    print tm.df.shape
    tm.df[predictors] = normalize(tm.df[predictors])

    df_train = tm.df[tm.df.index<date]
    df_test = tm.df[tm.df.index>=date]
    
    tm.X_train,tm.y_train = df_train[predictors].values,df_train[target].values
    tm.X_test, tm.y_test  = df_test[predictors].values,df_test[target].values  

In [14]:
train_test_split(tm,"2017-02-16","DOWN_next_Low_under_Close")

(11536, 36)
(11506, 36)


/home/ramon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ramon/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [15]:
tm.y_test

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0])

In [16]:
tm.X_test.shape

(11, 27)

In [76]:
X_train = tm.X_train
X_test = tm.X_test

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
y_train = pd.get_dummies(tm.y_train).values
y_test = pd.get_dummies(tm.y_test).values

In [ ]:
data_dim = 27
timesteps = 1
num_classes = 10
batch_size = 1

model = Sequential()

model = Sequential()
model.add(LSTM(32, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(32, return_sequences=True, stateful=True))
model.add(LSTM(32, stateful=True))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.fit(X_train, y_train, nb_epoch=10, batch_size=1, verbose=1);

Epoch 1/10
 6424/11495 [===============>..............] - ETA: 34s - loss: 0.6567 - acc: 0.6396

In [ ]:
loss, accuracy = model.evaluate(test_X, test_y_ohe, verbose=0)
print("Accuracy = {:.2f}".format(accuracy))